# 1. Import Libraries and Data

The molecular descriptors and fingerprints data was previously created in the `PI_data_loading.ipynb` notebook and has been stored in the data folder as csv file.

In [1]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler

# Models for scaled data
from sklearn.linear_model import ElasticNet, LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor

# Tree based models
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings('ignore')

In [2]:
pi = pd.read_csv('../data/pi_df_clean.csv')
print(pi.shape)
pi.head()

(834, 13)


,molecule_chembl_id,canonical_smiles,standard_type,standard_value,pchembl_value,assay_chembl_id,assay_description,target_chembl_id,target_pref_name,target_organism,standard_units_norm,IC50_nM,pIC50
0,CHEMBL1627209,O=C(N[C@@H]1c2ccccc2C[C@@H]1O)[C@@H](Cc1ccccc1...,IC50,21.1,7.68,CHEMBL899796,Inhibition of HIV1 recombinant protease,CHEMBL2366517,Protease,Human immunodeficiency virus 1,nM,21.1,7.675718
1,CHEMBL1627287,N=[S+]([O-])([C@H](Cc1ccccc1)C(=O)N[C@H]1c2ccc...,IC50,153.9,6.81,CHEMBL899796,Inhibition of HIV1 recombinant protease,CHEMBL2366517,Protease,Human immunodeficiency virus 1,nM,153.9,6.812761
2,CHEMBL1627235,N=[S+]([O-])([C@H](Cc1ccccc1)C(=O)N[C@H]1c2ccc...,IC50,37.3,7.43,CHEMBL899796,Inhibition of HIV1 recombinant protease,CHEMBL2366517,Protease,Human immunodeficiency virus 1,nM,37.3,7.428291
3,CHEMBL1627210,N=[S+]([O-])(C[C@H](Cc1ccccc1)C(=O)N[C@@H]1c2c...,IC50,2.5,8.60,CHEMBL899796,Inhibition of HIV1 recombinant protease,CHEMBL2366517,Protease,Human immunodeficiency virus 1,nM,2.5,8.602060
4,CHEMBL396814,O=C(N[C@H]1c2ccccc2C[C@H]1O)[C@@H](Cc1ccccc1)C...,IC50,10000.0,NaN,CHEMBL899796,Inhibition of HIV1 recombinant protease,CHEMBL2366517,Protease,Human immunodeficiency virus 1,nM,10000.0,5.000000


# Training QSAR Models

In [3]:
df = pd.read_csv('../data/pi_qsar_features.csv')
print("rows, cols:", df.shape)

rows, cols: (834, 523)


In [4]:
df.head()

,MolWt,MolLogP,MolMR,TPSA,NumHDonors,NumHAcceptors,NumRotatableBonds,NumAromaticRings,HeavyAtomCount,FractionCSP3,...,Morgan_512_503,Morgan_512_504,Morgan_512_505,Morgan_512_506,Morgan_512_507,Morgan_512_508,Morgan_512_509,Morgan_512_510,Morgan_512_511,pIC50
0,636.814,4.00190,179.2214,121.72,4,5,12,4,46,0.315789,...,0,0,0,0,1,0,0,0,0,7.675718
1,623.775,3.80477,172.8031,145.57,5,6,10,4,45,0.277778,...,0,0,0,0,1,0,0,0,0,6.812761
2,623.775,3.80477,172.8031,145.57,5,6,10,4,45,0.277778,...,0,0,0,0,1,0,0,0,0,7.428291
3,651.829,4.29997,181.9411,145.57,5,6,12,4,47,0.315789,...,0,0,0,0,1,0,0,0,0,8.602060
4,636.814,4.00190,179.2214,121.72,4,5,12,4,46,0.315789,...,0,0,0,0,1,0,0,0,0,5.000000


In [5]:
# Prepare X and y
y = df['pIC50']
X = df.drop(columns=['pIC50'])
print('Feature count: ', X.shape[1])

Feature count:  522


In [6]:
# ------------------------------------------------------------------------------
# Quick feature cleanup
# ------------------------------------------------------------------------------

# a. remove near-constant features

# Fit the selector
vt = VarianceThreshold(threshold=1e-6)
X_v = vt.fit_transform(X)
# Features kept
kept_features = X.columns[vt.get_support()].tolist()
# Features removed
removed_features = [col for col in X.columns if col not in kept_features]
# Create the reduced DataFrame
X = pd.DataFrame(X_v, columns=kept_features)

print(f"Features after variance filter: {len(kept_features)}")
print(f"Features removed: {len(removed_features)}")
# Optional: display or save the removed features
print("\nRemoved features (first 20):")
print(removed_features[:20])

Features after variance filter: 521
Features removed: 1

Removed features (first 20):
['Morgan_512_499']


In [7]:
# b. (optional) remove extremely collinear features (simple correlation filter)

corr_thresh = 0.98
corr_matrix = X.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [col for col in upper.columns if any(upper[col] > corr_thresh)]
if to_drop:
    X.drop(columns=to_drop, inplace=True)
    print("Dropped highly correlated features:", to_drop)

Dropped highly correlated features: ['MolMR', 'HeavyAtomCount']


In [8]:
X.shape

(834, 519)

In [9]:
# ------------------------------------------------------------------------------
# Define descriptor columns to scale and fingerprint columns to pass through
# ------------------------------------------------------------------------------
descriptors = ['MolWt', 'MolLogP', 'TPSA', 'NumHDonors', 'NumHAcceptors',
    'NumRotatableBonds', 'NumAromaticRings', 'FractionCSP3']
fingerprints = [c for c in X.columns if c not in descriptors]

# ColumnTransformer: scale descriptors, pass fingerprints through
preprocessor = ColumnTransformer([
    ("desc", StandardScaler(), descriptors),
    ("fp", "passthrough", fingerprints)
])

In [10]:
# ---------------------------
# k-fold OOF trainer
# ---------------------------
def kfold_train_predict(model, X_df, y_arr, n_splits=5, random_state=42):
    """
    model: estimator or pipeline (should accept fit/predict)
    X_df: pandas DataFrame (full dataset)
    y_arr: numpy array or Series
    returns: oof_preds (np.array same length as X_df), mean_r2, mean_rmse
    """
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    oof_preds = np.zeros(len(X_df))
    fold_scores = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(X_df, y_arr), 1):
        X_train, X_val = X_df.iloc[train_idx], X_df.iloc[val_idx]
        y_train, y_val = y_arr[train_idx], y_arr[val_idx]

        # fit on train fold, predict on val fold
        model.fit(X_train, y_train)
        preds = model.predict(X_val)

        oof_preds[val_idx] = preds

        fold_r2 = r2_score(y_val, preds)
        fold_rmse = np.sqrt(mean_squared_error(y_val, preds))
        fold_scores.append((fold_r2, fold_rmse))
        print(f"Fold {fold}: R2 = {fold_r2:.3f}, RMSE = {fold_rmse:.3f}")

    mean_r2 = np.mean([s[0] for s in fold_scores])
    mean_rmse = np.mean([s[1] for s in fold_scores])
    print(f"\nMean CV R2 = {mean_r2:.3f}, Mean RMSE = {mean_rmse:.3f}")
    return oof_preds, mean_r2, mean_rmse

In [11]:
# ---------------------------
# Define models and pipelines
# ---------------------------
models = {}

# Linear models (need scaling) -> include preprocessor
models['Linear'] = Pipeline([("preproc", preprocessor), ("model", LinearRegression())])
models['ElasticNet'] = Pipeline([("preproc", preprocessor), ("model", ElasticNet(random_state=42, max_iter=5000))])

# SVR (needs scaling)
models['SVM'] = Pipeline([("preproc", preprocessor), ("model", SVR())])

# KNN
models['KNN'] = Pipeline([("preproc", preprocessor), ("model", KNeighborsRegressor())])

# MLP
models['MLP'] = Pipeline([("preproc", preprocessor), ("model", MLPRegressor(max_iter=2000, random_state=42))])

# Random Forest (trees don't need scaling, but pipeline is fine)
models['Random Forest'] = Pipeline([("preproc", preprocessor), ("model", RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1))])

# XGBoost
models['XGBoost'] = Pipeline([("preproc", preprocessor), ("model", XGBRegressor(n_estimators=200, random_state=42, n_jobs=-1, objective='reg:squarederror'))])

In [12]:
# ---------------------------
# Run k-fold for each model
# ---------------------------
results = {}
oof_dict = {}

for name, model in models.items():
    print(f"\n=== Training model: {name} ===")
    oof_preds, mean_r2, mean_rmse = kfold_train_predict(model, X, y, n_splits=5, random_state=42)
    results[name] = {"r2": mean_r2, "rmse": mean_rmse}
    oof_dict[name] = oof_preds

# Summary
print("\nSummary (mean CV results):")
for name, res in results.items():
    print(f"{name}: R2 = {res['r2']:.4f}, RMSE = {res['rmse']:.4f}")


=== Training model: Linear ===
Fold 1: R2 = -1.433, RMSE = 2.736
Fold 2: R2 = -0.521, RMSE = 2.214
Fold 3: R2 = -1.164, RMSE = 2.549
Fold 4: R2 = -0.456, RMSE = 2.226
Fold 5: R2 = -0.955, RMSE = 2.571

Mean CV R2 = -0.906, Mean RMSE = 2.459

=== Training model: ElasticNet ===
Fold 1: R2 = 0.121, RMSE = 1.644
Fold 2: R2 = 0.085, RMSE = 1.717
Fold 3: R2 = 0.099, RMSE = 1.644
Fold 4: R2 = 0.118, RMSE = 1.732
Fold 5: R2 = 0.116, RMSE = 1.728

Mean CV R2 = 0.108, Mean RMSE = 1.693

=== Training model: SVM ===
Fold 1: R2 = 0.785, RMSE = 0.814
Fold 2: R2 = 0.777, RMSE = 0.849
Fold 3: R2 = 0.773, RMSE = 0.826
Fold 4: R2 = 0.768, RMSE = 0.889
Fold 5: R2 = 0.744, RMSE = 0.930

Mean CV R2 = 0.769, Mean RMSE = 0.862

=== Training model: KNN ===
Fold 1: R2 = 0.783, RMSE = 0.817
Fold 2: R2 = 0.720, RMSE = 0.950
Fold 3: R2 = 0.772, RMSE = 0.828
Fold 4: R2 = 0.781, RMSE = 0.863
Fold 5: R2 = 0.746, RMSE = 0.926

Mean CV R2 = 0.760, Mean RMSE = 0.877

=== Training model: MLP ===
Fold 1: R2 = 0.760, RMS

# 6. Tune the Best Models

### Support Vector Machine (SVR),
### K-Nearest Neighbours (KNN),
### Random Forest (RF)

In [13]:
import optuna
import joblib
from sklearn.decomposition import PCA

In [14]:
# PCA variant of the model
n_fp = len(fingerprints)
print(f"Using {len(descriptors)} descriptor columns and {n_fp} fingerprint columns.")

if n_fp == 0:
    raise ValueError("No fingerprint columns detected. Make sure your X contains fingerprint bit columns.")

# ---------- CV settings ----------
cv_outer = KFold(n_splits=5, shuffle=True, random_state=42)

Using 8 descriptor columns and 511 fingerprint columns.


In [15]:
# ----------------------- SVM ---------------------------------
def run_optuna_svm(n_trials=80):
    def objective(trial):
        use_pca = trial.suggest_categorical("use_pca", [True, False])
        if use_pca:
            max_comp = min(n_fp, 300)
            n_comp = trial.suggest_int("pca_n_components", 10, max(10, max_comp))
            fp_transform = Pipeline([("pca", PCA(n_components=n_comp, svd_solver="randomized", random_state=42))])
        else:
            fp_transform = "passthrough"

        preprocessor = ColumnTransformer([
            ("desc", StandardScaler(), descriptors),
            ("fp", fp_transform, fingerprints)
        ])

        # SVR hyperparameters (log-uniform sampling where appropriate)
        kernel = trial.suggest_categorical("kernel", ["rbf", "poly", "sigmoid"])
        C = trial.suggest_loguniform("C", 1e-3, 1e3)
        epsilon = trial.suggest_loguniform("epsilon", 1e-4, 1.0)
        gamma_choice = trial.suggest_categorical("gamma_choice", ["scale", "auto", "numeric"])
        if gamma_choice == "numeric":
            gamma = trial.suggest_loguniform("gamma", 1e-5, 1e1)
        else:
            gamma = gamma_choice
        # degree and coef0 for polynomial kernel
        if kernel == "poly":
            degree = trial.suggest_int("degree", 2, 5)
            coef0 = trial.suggest_float("coef0", 0.0, 1.0)
        else:
            degree = 3
            coef0 = 0.0

        svr = SVR(kernel=kernel, C=C, epsilon=epsilon, gamma=gamma, degree=degree, coef0=coef0, max_iter=100000)
        pipe = Pipeline([("preproc", preprocessor), ("svr", svr)])
        scores = cross_val_score(pipe, X, y, cv=cv_outer, scoring="r2", n_jobs=-1)
        return float(np.mean(scores))

    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(),
                                pruner=optuna.pruners.MedianPruner())
    study.optimize(objective, n_trials=n_trials, n_jobs=1, show_progress_bar=True)

    # Rebuild best pipeline and fit on full data
    best = study.best_params
    if best.get("use_pca", False):
        fp_transform = Pipeline([("pca", PCA(n_components=best["pca_n_components"], random_state=42))])
    else:
        fp_transform = "passthrough"
    preprocessor_best = ColumnTransformer([
        ("desc", StandardScaler(), descriptors),
        ("fp", fp_transform, fingerprints)
    ])
    # reconstruct SVR
    best_kernel = best.get("kernel", "rbf")
    best_gamma = best.get("gamma", best.get("gamma_choice", "scale"))
    if best_gamma == "numeric":
        best_gamma = best.get("gamma")
    best_degree = best.get("degree", 3)
    best_coef0 = best.get("coef0", 0.0)

    best_svm = SVR(
        kernel=best_kernel,
        C=best.get("C"),
        epsilon=best.get("epsilon"),
        gamma=best_gamma,
        degree=best_degree,
        coef0=best_coef0,
        max_iter=100000
    )
    best_pipe = Pipeline([("preproc", preprocessor_best), ("svr", best_svm)])
    best_pipe.fit(X, y)
    joblib.dump(best_pipe, "../models/pi_svm.joblib")
    print("SVM best R2:", study.best_value)
    print("SVM best params:", study.best_params)
    return study, best_pipe

In [16]:
# ------------------ KNN -----------------------
def run_optuna_knn(n_trials=50):
    def objective(trial):
        use_pca = trial.suggest_categorical("use_pca", [True, False])
        if use_pca:
            max_comp = min(n_fp, 300)
            n_comp = trial.suggest_int("pca_n_components", 10, max(10, max_comp))
            fp_transform = Pipeline([("pca", PCA(n_components=n_comp, svd_solver="randomized", random_state=42))])
        else:
            fp_transform = "passthrough"

        preprocessor = ColumnTransformer([
            ("desc", StandardScaler(), descriptors),
            ("fp", fp_transform, fingerprints)
        ])

        n_neighbors = trial.suggest_int("n_neighbors", 1, 30)
        weights = trial.suggest_categorical("weights", ["uniform", "distance"])
        p = trial.suggest_int("p", 1, 2)  # 1 = manhattan, 2 = euclidean

        knn = KNeighborsRegressor(n_neighbors=n_neighbors, weights=weights, p=p)
        pipe = Pipeline([("preproc", preprocessor), ("knn", knn)])
        scores = cross_val_score(pipe, X, y, cv=cv_outer, scoring="r2", n_jobs=-1)
        return float(np.mean(scores))

    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(),
                                pruner=optuna.pruners.MedianPruner())
    study.optimize(objective, n_trials=n_trials, n_jobs=1, show_progress_bar=True)

    # Rebuild best pipeline and fit on full data
    best = study.best_params
    if best.get("use_pca", False):
        fp_transform = Pipeline([("pca", PCA(n_components=best["pca_n_components"], random_state=42))])
    else:
        fp_transform = "passthrough"
    preprocessor_best = ColumnTransformer([
        ("desc", StandardScaler(), descriptors),
        ("fp", fp_transform, fingerprints)
    ])
    best_knn = KNeighborsRegressor(n_neighbors=best["n_neighbors"], weights=best["weights"], p=best["p"])
    best_pipe = Pipeline([("preproc", preprocessor_best), ("knn", best_knn)])
    best_pipe.fit(X, y)
    joblib.dump(best_pipe, "../models/pi_knn.joblib")
    print("KNN best R2:", study.best_value)
    print("KNN best params:", study.best_params)
    return study, best_pipe

In [17]:
# -------------------- Random Forest --------------------------------
def run_optuna_rf(n_trials=50):
    def objective(trial):
        preprocessor = ColumnTransformer([
            ("desc", StandardScaler(), descriptors),
            ("fp", "passthrough", fingerprints)
        ])

        n_estimators = trial.suggest_int("n_estimators", 100, 500)
        max_depth = trial.suggest_int("max_depth", 3, 50)
        min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
        min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 20)
        max_features = trial.suggest_categorical("max_features", ["sqrt", "log2", 0.3, 0.5, 0.8])

        rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth,
                                   min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf,
                                   max_features=max_features, n_jobs=-1, random_state=42)
        pipe = Pipeline([("preproc", preprocessor), ("rf", rf)])
        scores = cross_val_score(pipe, X, y, cv=cv_outer, scoring="r2", n_jobs=-1)
        return float(np.mean(scores))

    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(),
                                pruner=optuna.pruners.MedianPruner())
    study.optimize(objective, n_trials=n_trials, n_jobs=1, show_progress_bar=True)

    best = study.best_params
    preprocessor_best = ColumnTransformer([
        ("desc", StandardScaler(), descriptors),
        ("fp", "passthrough", fingerprints)
    ])
    best_rf = RandomForestRegressor(n_estimators=best["n_estimators"], max_depth=best["max_depth"],
                                    min_samples_split=best["min_samples_split"],
                                    min_samples_leaf=best["min_samples_leaf"],
                                    max_features=best["max_features"], n_jobs=-1, random_state=42)
    best_pipe = Pipeline([("preproc", preprocessor_best), ("rf", best_rf)])
    best_pipe.fit(X, y)
    joblib.dump(best_pipe, "../models/pi_rf.joblib")
    print("RF best R2:", study.best_value)
    print("RF best params:", study.best_params)
    return study, best_pipe

In [18]:
# ----------------------------
# Example: run all studies sequentially (adjust n_trials)
# ----------------------------
svm_study, svm_pipe = run_optuna_svm(n_trials=50)

[I 2025-10-07 08:43:50,993] A new study created in memory with name: no-name-e524b7f5-bed2-4444-9f40-ca9cd4eba564
Best trial: 0. Best value: 0.745879:   2%|▏         | 1/50 [00:05<04:47,  5.88s/it]

[I 2025-10-07 08:43:56,873] Trial 0 finished with value: 0.7458786566524909 and parameters: {'use_pca': True, 'pca_n_components': 128, 'kernel': 'rbf', 'C': 0.45487253388891086, 'epsilon': 0.0006808829161516052, 'gamma_choice': 'scale'}. Best is trial 0 with value: 0.7458786566524909.


Best trial: 0. Best value: 0.745879:   4%|▍         | 2/50 [00:09<03:37,  4.53s/it]

[I 2025-10-07 08:44:00,447] Trial 1 finished with value: 0.7193074734075713 and parameters: {'use_pca': True, 'pca_n_components': 122, 'kernel': 'sigmoid', 'C': 8.129760396229347, 'epsilon': 0.0257863509804728, 'gamma_choice': 'auto'}. Best is trial 0 with value: 0.7458786566524909.


Best trial: 0. Best value: 0.745879:   6%|▌         | 3/50 [00:10<02:08,  2.73s/it]

...

[I 2025-10-07 08:44:23,944] Trial 47 finished with value: 0.7786121818830714 and parameters: {'use_pca': True, 'pca_n_components': 263, 'kernel': 'rbf', 'C': 26.07236136884119, 'epsilon': 0.18687299567064683, 'gamma_choice': 'auto'}. Best is trial 44 with value: 0.7824509015948344.


Best trial: 44. Best value: 0.782451:  98%|█████████▊| 49/50 [00:33<00:00,  1.87it/s]

[I 2025-10-07 08:44:24,490] Trial 48 finished with value: 0.7724517237101943 and parameters: {'use_pca': True, 'pca_n_components': 264, 'kernel': 'rbf', 'C': 52.14914333768239, 'epsilon': 0.15716963821242436, 'gamma_choice': 'auto'}. Best is trial 44 with value: 0.7824509015948344.


Best trial: 44. Best value: 0.782451: 100%|██████████| 50/50 [00:33<00:00,  1.47it/s]


[I 2025-10-07 08:44:24,980] Trial 49 finished with value: 0.6716591784959041 and parameters: {'use_pca': True, 'pca_n_components': 265, 'kernel': 'poly', 'C': 25.043559470483444, 'epsilon': 0.8057782247721512, 'gamma_choice': 'auto', 'degree': 3, 'coef0': 0.07562095708048178}. Best is trial 44 with value: 0.7824509015948344.
SVM best R2: 0.7824509015948344
SVM best params: {'use_pca': True, 'pca_n_components': 220, 'kernel': 'rbf', 'C': 10.629746378833838, 'epsilon': 0.10229560218392462, 'gamma_choice': 'auto'}


In [19]:
knn_study, knn_pipe = run_optuna_knn(n_trials=50)

[I 2025-10-07 08:44:25,373] A new study created in memory with name: no-name-04233f45-8a87-4354-abe9-8f968a4d25ba
Best trial: 0. Best value: 0.713689:   2%|▏         | 1/50 [00:00<00:17,  2.74it/s]

[I 2025-10-07 08:44:25,733] Trial 0 finished with value: 0.713689233868753 and parameters: {'use_pca': True, 'pca_n_components': 28, 'n_neighbors': 2, 'weights': 'distance', 'p': 2}. Best is trial 0 with value: 0.713689233868753.


Best trial: 1. Best value: 0.736402:   4%|▍         | 2/50 [00:00<00:22,  2.12it/s]

[I 2025-10-07 08:44:26,276] Trial 1 finished with value: 0.7364020842677571 and parameters: {'use_pca': True, 'pca_n_components': 227, 'n_neighbors': 7, 'weights': 'distance', 'p': 2}. Best is trial 1 with value: 0.7364020842677571.


Best trial: 1. Best value: 0.736402:   6%|▌         | 3/50 [00:01<00:17,  2.76it/s]

...

[I 2025-10-07 08:44:41,995] Trial 47 finished with value: 0.6920749209522636 and parameters: {'use_pca': False, 'n_neighbors': 27, 'weights': 'uniform', 'p': 1}. Best is trial 33 with value: 0.7659725043609967.


Best trial: 33. Best value: 0.765973:  98%|█████████▊| 49/50 [00:16<00:00,  3.21it/s]

[I 2025-10-07 08:44:42,281] Trial 48 finished with value: 0.7602692173276665 and parameters: {'use_pca': False, 'n_neighbors': 9, 'weights': 'uniform', 'p': 1}. Best is trial 33 with value: 0.7659725043609967.


Best trial: 33. Best value: 0.765973: 100%|██████████| 50/50 [00:17<00:00,  2.90it/s]

[I 2025-10-07 08:44:42,597] Trial 49 finished with value: 0.7142793449704775 and parameters: {'use_pca': False, 'n_neighbors': 21, 'weights': 'uniform', 'p': 1}. Best is trial 33 with value: 0.7659725043609967.
KNN best R2: 0.7659725043609967
KNN best params: {'use_pca': False, 'n_neighbors': 3, 'weights': 'uniform', 'p': 1}


In [20]:
rf_study, rf_pipe = run_optuna_rf(n_trials=50)

[I 2025-10-07 08:44:42,675] A new study created in memory with name: no-name-0267da4e-dad1-4765-8812-ac60547e6861
Best trial: 0. Best value: 0.727811:   2%|▏         | 1/50 [00:02<02:22,  2.90s/it]

[I 2025-10-07 08:44:45,572] Trial 0 finished with value: 0.7278113040569016 and parameters: {'n_estimators': 253, 'max_depth': 26, 'min_samples_split': 17, 'min_samples_leaf': 12, 'max_features': 0.8}. Best is trial 0 with value: 0.7278113040569016.


Best trial: 0. Best value: 0.727811:   4%|▍         | 2/50 [00:05<02:00,  2.50s/it]

[I 2025-10-07 08:44:47,796] Trial 1 finished with value: 0.7030440733361011 and parameters: {'n_estimators': 434, 'max_depth': 47, 'min_samples_split': 2, 'min_samples_leaf': 18, 'max_features': 0.3}. Best is trial 0 with value: 0.7278113040569016.


Best trial: 2. Best value: 0.748577:   6%|▌         | 3/50 [00:09<02:35,  3.32s/it]

...

[I 2025-10-07 08:46:57,694] Trial 47 finished with value: 0.7720725097645643 and parameters: {'n_estimators': 469, 'max_depth': 14, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 0.3}. Best is trial 35 with value: 0.7730782056657596.


Best trial: 35. Best value: 0.773078:  98%|█████████▊| 49/50 [02:21<00:04,  4.41s/it]

[I 2025-10-07 08:47:03,969] Trial 48 finished with value: 0.7721923904804227 and parameters: {'n_estimators': 481, 'max_depth': 14, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 0.3}. Best is trial 35 with value: 0.7730782056657596.


Best trial: 35. Best value: 0.773078: 100%|██████████| 50/50 [02:46<00:00,  3.33s/it]


[I 2025-10-07 08:47:29,018] Trial 49 finished with value: 0.7547027128762174 and parameters: {'n_estimators': 477, 'max_depth': 14, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_features': 0.8}. Best is trial 35 with value: 0.7730782056657596.
RF best R2: 0.7730782056657596
RF best params: {'n_estimators': 380, 'max_depth': 23, 'min_samples_split': 8, 'min_samples_leaf': 2, 'max_features': 0.3}


# 7. Try a meta model or ensembler of the best models

In [21]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_predict

# reproducible folds
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# 1) Get OOF predictions for each base model (these preds are made by models
#    that were trained without the corresponding sample — no leakage)
print("Generating OOF preds (this may take time)...")
oof_knn = cross_val_predict(knn_pipe, X, y, cv=kf, n_jobs=-1, method='predict')
oof_svm = cross_val_predict(svm_pipe, X, y, cv=kf, n_jobs=-1, method='predict')
oof_rf  = cross_val_predict(rf_pipe,  X, y, cv=kf, n_jobs=-1, method='predict')

# Stack OOF predictions (n_samples x n_models)
stack_oof = np.vstack([oof_knn, oof_svm, oof_rf]).T

# 2) Simple average ensemble
ens_mean = stack_oof.mean(axis=1)
r2_mean = r2_score(y, ens_mean)
rmse_mean = np.sqrt(mean_squared_error(y, ens_mean))
print(f"Simple average ensemble -> R2: {r2_mean:.4f}, RMSE: {rmse_mean:.4f}")

Generating OOF preds (this may take time)...
Simple average ensemble -> R2: 0.7925, RMSE: 0.8179


In [22]:
# 3) Stacking: train a Ridge meta-learner on the OOF stack
meta = Ridge(alpha=1.0)
meta.fit(stack_oof, y)                 # training on OOF preds is OK (no leakage)
ens_stack = meta.predict(stack_oof)    # predictions on the same OOF matrix
r2_stack = r2_score(y, ens_stack)
rmse_stack = np.sqrt(mean_squared_error(y, ens_stack))
print(f"Stacking (Ridge) ensemble -> R2: {r2_stack:.4f}, RMSE: {rmse_stack:.4f}")

# Show meta-learner weights
print("Meta-learner coefficients (weights):", meta.coef_)
print("Meta-learner intercept:", meta.intercept_)

Stacking (Ridge) ensemble -> R2: 0.7945, RMSE: 0.8141
Meta-learner coefficients (weights): [0.34952213 0.54508747 0.0954718 ]
Meta-learner intercept: 0.057185963824212926


In [23]:
joblib.dump(meta, "../models/pi_meta.joblib")
print("Saved final base pipelines and meta-learner.")

Saved final base pipelines and meta-learner.
